# **1) Import all packages**

In [2]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import math
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split 
from numpy.random import seed
from sklearn.preprocessing import LabelEncoder
seed(42)
from sklearn.metrics import roc_auc_score
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K
from keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.utils import class_weight
tf.random.set_seed(42)

# **2) Get Dataset**

In [3]:
df = pd.read_excel('/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Data/default of credit card clients.xls', index_col=0)

In [4]:
print(df.head())
print(df.shape)
print(df.shape)

           X1   X2         X3  ...       X22       X23                           Y
ID  LIMIT_BAL  SEX  EDUCATION  ...  PAY_AMT5  PAY_AMT6  default payment next month
1       20000    2          2  ...         0         0                           1
2      120000    2          2  ...         0      2000                           1
3       90000    2          2  ...      1000      5000                           0
4       50000    2          2  ...      1069      1000                           0

[5 rows x 24 columns]
(30001, 24)
(30001, 24)


In [5]:
print(df.columns)

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21',
       'X22', 'X23', 'Y'],
      dtype='object')


## **3) Prepare Data**

In [6]:
df['Y'].value_counts()

0                             23364
1                              6636
default payment next month        1
Name: Y, dtype: int64

In [7]:
 x = df.isna().sum()
 print(x)

X1     0
X2     0
X3     0
X4     0
X5     0
X6     0
X7     0
X8     0
X9     0
X10    0
X11    0
X12    0
X13    0
X14    0
X15    0
X16    0
X17    0
X18    0
X19    0
X20    0
X21    0
X22    0
X23    0
Y      0
dtype: int64


In [8]:
#get labels
labels = df.pop('Y')

In [9]:
labels_array = labels[1:len(labels)].to_numpy() 
print(labels_array)
labelsv2 = np.vstack(labels_array)
print(labelsv2)
labels[1:len(labels)].value_counts(normalize=True) * 100

[1 1 0 ... 1 1 1]
[[1]
 [1]
 [0]
 ...
 [1]
 [1]
 [1]]


0    77.88
1    22.12
Name: Y, dtype: float64

In [10]:
#Split data
X_train, X_test, y_train, y_test =  train_test_split(df[1:len(df)], labelsv2, train_size=0.85, test_size=0.15, random_state=42)

In [11]:
#Scale Data
# Training Data
X_train = MinMaxScaler().fit_transform(X_train)

# Testing Data
X_test = MinMaxScaler().fit_transform(X_test)

In [12]:
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weights_train.npy",X_train)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weigths_trainlabels.npy",y_train)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weights_test.npy",X_test)
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weigths_testlabels.npy",y_test)

# **4) Vanilla Decision Tree**

In [13]:
#Create decision tree for specified depth of 5 and 10 
model = DecisionTreeClassifier(max_depth=5,random_state=16)
model.fit(X_train,y_train)
#max_depth=10,

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=16, splitter='best')

In [14]:
res_pred = model.predict(X_test)

In [15]:
#Get various scores
acc = accuracy_score(y_test, res_pred)
rec = recall_score(y_test, res_pred,average='macro')
prec = precision_score(y_test, res_pred,average='macro')
auc = roc_auc_score(y_test,res_pred, average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.8133333333333334
Recall:  0.6324518010167381
Precision: 0.7544076197402222
AUC:  0.6324518010167381


# **5) Baseline Model**

In [16]:
#Load data for re-run if needed
X_train = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weights_train.npy")
y_train = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weigths_trainlabels.npy")
X_test = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weights_test.npy")
y_test = np.load("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weigths_testlabels.npy")

In [17]:
log_X_train = X_train
log_y_train = y_train
log_X_test = X_test
log_y_test = y_test

In [18]:
#Create Logistic Regression Model
logreg = LogisticRegression(max_iter= 3000,random_state=16)
logreg.fit(log_X_train,log_y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=16, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
print(logit_roc_auc)

0.6388397005152686


# **6) Built Neural Network**

In [20]:
print(X_train.shape)
print(X_train)
print(y_train.shape)
X_train = np.asarray(X_train).astype('float32')

(25500, 23)
[[0.02020202 1.         0.33333333 ... 0.00540258 0.0026868  0.        ]
 [0.16161616 0.         0.66666667 ... 0.         0.01149277 0.01138211]
 [0.33333333 0.         0.16666667 ... 0.00309179 0.02145458 0.35474245]
 ...
 [0.04040404 0.         0.16666667 ... 0.         0.         0.        ]
 [0.06060606 1.         0.33333333 ... 0.00193237 0.         0.00227642]
 [0.15151515 1.         0.33333333 ... 0.002657   0.03329199 0.00284553]]
(25500, 1)


In [21]:
print(X_test.shape)
print(y_test.shape)
print(y_test)
X_test = np.asarray(X_test).astype('float32')

(4500, 23)
(4500, 1)
[[0]
 [0]
 [0]
 ...
 [0]
 [1]
 [0]]


In [22]:
y_train = y_train.flatten()
print(y_train.shape)

(25500,)


In [23]:
#Set class weights
weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weight_dict = dict(enumerate(weight))

In [24]:
#Prepare labels for Neural Network
y_train = np.array(keras.utils.to_categorical(y_train))
y_test = np.array(keras.utils.to_categorical(y_test))
print(y_train.shape)
print(y_train[:10])

(25500, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [25]:
#Metrics for Neural Network
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

val_auc = tf.keras.metrics.AUC()

In [26]:
#Function to built model
def build_model():
  inputs = layers.Input(shape=(23))

  x = layers.Dense(23,activation='relu',name='fc1')(inputs)

  x = layers.Dense(20,activation='relu',name='fc2')(x)

  x = layers.Dense(10,activation='relu',name='fc3')(x)

  x = layers.Dense(2,name='logits')(x)

  preds = layers.Activation('softmax',name='Softmax')(x)

  model = Model(inputs=inputs, outputs=preds)
  model.summary()
  return model
model = build_model()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         (None, 2)                 0         
Total params: 1,264
Trainable params: 1,264
Non-trainable params: 0
____________________________________________________

In [27]:
#Saving best performing model
filepath = "/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Preset Models/ModelV6.hdf5"

In [28]:
#Set-up filepath and ModelCheckepoint with Earlystopping to save best performing moddel based on Validation AUC
model_checkpoint = ModelCheckpoint(filepath,  monitor='val_auc', save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_auc', mode='max', verbose=0, patience=5)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC(name='auc')])

In [29]:
#Set epochs and bach size, and train model 
num_epoch = 100
batch_size = 64
model1 = model.fit(x=X_train, y=y_train, epochs=num_epoch, batch_size=batch_size, validation_split=0.2,callbacks=[model_checkpoint,es])

Epoch 1/100
319/319 [==============================] - 2s 5ms/step - loss: 0.5086 - acc: 0.7855 - f1_m: 0.7855 - precision_m: 0.7855 - recall_m: 0.7855 - auc: 0.8236 - val_loss: 0.4687 - val_acc: 0.8029 - val_f1_m: 0.8028 - val_precision_m: 0.8028 - val_recall_m: 0.8028 - val_auc: 0.8548
Epoch 2/100
319/319 [==============================] - 1s 2ms/step - loss: 0.4591 - acc: 0.8084 - f1_m: 0.8084 - precision_m: 0.8084 - recall_m: 0.8084 - auc: 0.8590 - val_loss: 0.4580 - val_acc: 0.8104 - val_f1_m: 0.8103 - val_precision_m: 0.8103 - val_recall_m: 0.8103 - val_auc: 0.8617
Epoch 3/100
319/319 [==============================] - 1s 2ms/step - loss: 0.4509 - acc: 0.8108 - f1_m: 0.8108 - precision_m: 0.8108 - recall_m: 0.8108 - auc: 0.8656 - val_loss: 0.4521 - val_acc: 0.8127 - val_f1_m: 0.8126 - val_precision_m: 0.8126 - val_recall_m: 0.8126 - val_auc: 0.8642
Epoch 4/100
319/319 [==============================] - 1s 2ms/step - loss: 0.4466 - acc: 0.8140 - f1_m: 0.8139 - precision_m: 0.8139 

In [30]:
#Train Performance
prediction_model = tf.keras.models.load_model(filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(X_train, y_train, verbose=0, batch_size=64)

print('Train Accuracy:',accuracy)
print('Train Recall:',recall)
print('Train Precision:',precision)
print('Train F1:',f1_score)
print('Train AUC:',auc)
prediction_model.summary()

Train Accuracy: 0.8226274251937866
Train Recall: 0.8225250840187073
Train Precision: 0.8225250840187073
Train F1: 0.8225249648094177
Train AUC: 0.8819910287857056
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)   

In [31]:
#Test Performance
#filepath = "/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/CC Models/ModelV4.hdf5"
loss, accuracy, f1_score, precision, recall, auc = prediction_model.evaluate(X_test, y_test, verbose=0, batch_size=64)

print('Test Accuracy:',accuracy)
print('Test Recall:',recall)
print('Test Precision:',precision)
print('Test F1:',f1_score)
print('Test AUC:',auc)
prediction_model.summary()

Test Accuracy: 0.804888904094696
Test Recall: 0.8053256869316101
Test Precision: 0.8053256869316101
Test F1: 0.8053256273269653
Test AUC: 0.8738144636154175
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         

In [32]:
#Best Performance
best_filepath = "/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Preset Models/ModelV2.hdf5"
best_model = tf.keras.models.load_model(best_filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m, 'auc': tf.keras.metrics.AUC() }) 
loss, accuracy, f1_score, precision, recall, auc = best_model.evaluate(X_test, y_test, verbose=0, batch_size=64)

#print('Best Accuracy:',accuracy)
#print('Best Recall:',recall)
#print('Best Precision:',precision)
#print('Best F1:',f1_score)
print('Best AUC:',auc)
best_model.summary()

Best AUC: 0.8760309815406799
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         (None, 2)                 0         
Total params: 1,264
Trainable params: 1,264
Non-trainable params: 0
_______________________

# **7) Generate Soft Targets**

In [33]:
#Load the best model
model = tf.keras.models.load_model(best_filepath, custom_objects={'f1_m':f1_m,'precision_m':precision_m, 'recall_m':recall_m}) 

In [34]:
#Compile it again
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc',f1_m,precision_m, recall_m,tf.keras.metrics.AUC()])
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
fc1 (Dense)                  (None, 23)                552       
_________________________________________________________________
fc2 (Dense)                  (None, 20)                480       
_________________________________________________________________
fc3 (Dense)                  (None, 10)                210       
_________________________________________________________________
logits (Dense)               (None, 2)                 22        
_________________________________________________________________
Softmax (Activation)         (None, 2)                 0         
Total params: 1,264
Trainable params: 1,264
Non-trainable params: 0
____________________________________________________

In [35]:
#Remove last softmax layer and predict again
model_sans_softmax = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = model_sans_softmax.predict(X_train)

In [36]:
#Get the logits
soft_targets = model_logits

In [37]:
#Save if needed for later use
#np.save("/content/drive/My Drive/Tilburg University/DS&S/Thesis/Google Collab/Default Credit Card Clients/Preset Weights/Data [Preset Weights]/weights_Logits.npy", model_logits)

# **8) Knowledge Distillation Decision Tree**


In [38]:
#Set up Decision Tree for Knowledge Distillation at given depth 
model = DecisionTreeRegressor(max_depth=10)

In [39]:
#Train Decision Tree
model.fit(X_train,soft_targets)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=10,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [40]:
#Get predictions
res_pred = model.predict(X_test)

In [41]:
# Apply softmax function to output of Decision Tree
pred_argmax = np.zeros_like(res_pred)
pred_argmax[np.arange(len(res_pred)), res_pred.argmax(1)] = 1 

In [42]:
#Get scores of student decision tree
acc = accuracy_score(pred_argmax,y_test)
rec = recall_score(y_test, pred_argmax,average='macro')
prec = precision_score(y_test, pred_argmax,average='macro')
auc = roc_auc_score(y_test, pred_argmax,average='macro')
print('Accuracy:', acc)
print('Recall: ', rec)
print('Precision:', prec)
print('AUC: ', auc)

Accuracy: 0.8153333333333334
Recall:  0.6614695518819578
Precision: 0.7427161956221747
AUC:  0.6614695518819578
